 # Problem
We will have multiple tables in the database, and sometimes there won’t be a common “ID” or “KEY” to join them – scenarios like the following:
    • Customer information scattered across multiple tables and systems.
    • No global key to link them all together.
    • A lot of variations in names and addresses.

# Solution
This can be solved by applying text similarity functions on the demographic’s columns like the first name, last name, address, etc. And based on the similarity score on a few common columns, we can decide either the record pair is a match or not a match

# Deduplication in the same table

In [2]:
# !pip install recordlinkage

In [3]:
# Import package
import recordlinkage
#For this demo let us use the inbuilt dataset from recordlinkage library

In [4]:
#import data set
from recordlinkage.datasets import load_febrl1
#create a dataframe - dfa
dfA = load_febrl1()
dfA.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-223-org,NaN,waller,6,tullaroop street,willaroo,st james,4011,wa,19081209,6988048
rec-122-org,lachlan,berry,69,giblin street,killarney,bittern,4814,qld,19990219,7364009
rec-373-org,deakin,sondergeld,48,goldfinch circuit,kooltuo,canterbury,2776,vic,19600210,2635962
rec-10-dup-0,kayla,harrington,NaN,maltby circuit,coaling,coolaroo,3465,nsw,19150612,9004242
rec-227-org,luke,purdon,23,ramsay place,mirani,garbutt,2260,vic,19831024,8099933


In [5]:
#  let’s build the pairs using the first name as the blocking index.
indexer = recordlinkage.BlockIndex(on='given_name')
pairs = indexer.index(dfA)
print (len(pairs))

2082


In [6]:
# This cell can take some time to compute.
compare_cl = recordlinkage.Compare()
compare_cl.string('given_name', 'given_name',method='jarowinkler', label='given_name')
compare_cl.string('surname', 'surname', method='jarowinkler', label='surname')
compare_cl.exact('date_of_birth', 'date_of_birth', label='date_of_birth')
compare_cl.exact('suburb', 'suburb', label='suburb')
compare_cl.exact('state', 'state', label='state')
compare_cl.string('address_1', 'address_1',method='jarowinkler',label='address_1')
features = compare_cl.compute(pairs, dfA)
features.sample(5)

,,given_name,surname,date_of_birth,suburb,state,address_1
rec_id_1,rec_id_2,,,,,,
rec-32-org,rec-70-org,1.0,0.619048,0,0,0,0.518926
rec-127-dup-0,rec-468-dup-0,1.0,0.437037,0,0,0,0.506536
rec-77-dup-0,rec-382-org,1.0,0.576190,0,0,0,0.493715
rec-38-dup-0,rec-260-org,1.0,0.611111,0,0,0,0.586869
rec-369-org,rec-412-org,1.0,0.511905,0,0,0,0.624008


In [7]:
# select all the features except for given_name since its our blocking key
features1 = features[['suburb','state','surname','date_of_birth','address_1']]
# Unsupervised learning – probabilistic
ecm = recordlinkage.ECMClassifier()
result_ecm = ecm.learn((features1).astype(int),return_type ='series')
result_ecm

rec_id_1       rec_id_2     
rec-183-dup-0  rec-122-org      0
rec-248-org    rec-122-org      0
               rec-183-dup-0    0
rec-122-dup-0  rec-122-org      1
               rec-183-dup-0    0
                               ..
rec-252-org    rec-252-dup-0    1
rec-48-dup-0   rec-48-org       1
rec-298-dup-0  rec-298-org      1
rec-282-org    rec-282-dup-0    1
rec-327-org    rec-411-org      0
Name: classification, Length: 2082, dtype: int32

# deduplication in multiple tables

In [8]:
from recordlinkage.datasets import load_febrl4
dfA, dfB = load_febrl4()
dfB.head()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec_id,,,,,,,,,,
rec-561-dup-0,elton,NaN,3,light setreet,pinehill,windermere,3212,vic,19651013,1551941
rec-2642-dup-0,mitchell,maxon,47,edkins street,lochaoair,north ryde,3355,nsw,19390212,8859999
rec-608-dup-0,NaN,white,72,lambrigg street,kelgoola,broadbeach waters,3159,vic,19620216,9731855
rec-3239-dup-0,elk i,menzies,1,lyster place,NaN,northwood,2585,vic,19980624,4970481
rec-2886-dup-0,NaN,garanggar,NaN,may maxwell crescent,springettst arcade,forest hill,2342,vic,19921016,1366884


In [9]:
indexer = recordlinkage.BlockIndex(on='given_name')
pairs = indexer.index(dfA, dfB)

In [10]:
compare_cl = recordlinkage.Compare()
compare_cl.string('given_name', 'given_name',method='jarowinkler',label='given_name')
compare_cl.string('surname', 'surname', method='jarowinkler',label='surname')
compare_cl.exact('date_of_birth', 'date_of_birth', label='date_of_birth')
compare_cl.exact('suburb', 'suburb', label='suburb')
compare_cl.exact('state', 'state', label='state')
compare_cl.string('address_1', 'address_1',method='jarowinkler',label='address_1')
features = compare_cl.compute(pairs, dfA, dfB)
features.head(10)

given_name   surname  date_of_birth  suburb  \
rec_id_1     rec_id_2                                                      
rec-1070-org rec-3024-dup-0         1.0  0.436508              0       0   
             rec-2371-dup-0         1.0  0.490079              0       0   
             rec-4652-dup-0         1.0  0.490079              0       0   
             rec-4795-dup-0         1.0  0.000000              0       0   
             rec-1314-dup-0         1.0  0.000000              0       0   
rec-2371-org rec-3024-dup-0         1.0  0.527778              0       0   
             rec-2371-dup-0         1.0  1.000000              1       1   
             rec-4652-dup-0         1.0  0.500000              0       0   
             rec-4795-dup-0         1.0  0.527778              0       0   
             rec-1314-dup-0         1.0  0.527778              0       0   

                             state  address_1  
rec_id_1     rec_id_2                          
rec-1070-org rec-3024-dup-0      1   0.000000  
             rec-2371-dup-0      0   0.715873  
             rec-4652-dup-0      0   0.645604  
             rec-4795-dup-0      1   0.552381  
             rec-1314-dup-0      1   0.618254  
rec-2371-org rec-3024-dup-0      0   0.000000  
             rec-2371-dup-0      1   1.000000  
             rec-4652-dup-0      1   0.635684  
             rec-4795-dup-0      0   0.411111  
             rec-1314-dup-0      0   0.672222

"""
ECM – classifier : is an unsupervised learning method to calculate the probability that the record is a match.
""" 

In [11]:
# select all the features except for given_name since its our blocking key
features1 = features[['suburb','state','surname','date_of_birth','address_1']]
# unsupervised learning - probablistic
ecm = recordlinkage.ECMClassifier()
result_ecm = ecm.learn((features1).astype(int),return_type ='series')
result_ecm

rec_id_1      rec_id_2      
rec-1070-org  rec-3024-dup-0    0
              rec-2371-dup-0    0
              rec-4652-dup-0    0
              rec-4795-dup-0    0
              rec-1314-dup-0    0
                               ..
rec-4528-org  rec-4528-dup-0    1
rec-4887-org  rec-4887-dup-0    1
rec-4350-org  rec-4350-dup-0    1
rec-4569-org  rec-4569-dup-0    1
rec-3125-org  rec-3125-dup-0    1
Name: classification, Length: 77249, dtype: int32